In [ ]:
import sklearn
from sklearn.impute import SimpleImputer
from tensorflow import keras as k
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# %pip install d2l==1.0.0a1.post0
from d2l import tensorflow as d2l


In [ ]:
data = pd.read_csv('./input/dataset.csv')
# dataImputed = pd.read_csv('./input/datasetImputed.csv')

In [ ]:
# è stata riscritta una funzione di reshape inutile 
# e.g. (a_prev=x_train, season=trn_ssn)
def reshape_to_inputshape(a_prev,season):
    totalMatches = len(season)*38
    input_step = int(a_prev.shape[0]/totalMatches)
    prev_f = a_prev.shape[1]
    return np.reshape(a_prev, (totalMatches, input_step, prev_f))

In [ ]:
features = ['HomeTeam', 'AwayTeam', 
            'HTeamEloScore', 'ATeamEloScore', 
            'HTdaysSinceLastMatch', 'ATdaysSinceLastMatch', 
            'HTW_rate', 'ATW_rate', 'ATD_rate', 'HTD_rate', 
            '7_HTW_rate', '12_HTW_rate', '7_ATW_rate', '12_ATW_rate', 
            '7_HTD_rate', '12_HTD_rate', '7_ATD_rate', '12_ATD_rate',
            '7_HTL_rate', '12_HTL_rate', '7_ATL_rate', '12_ATL_rate',
            '5_HTHW_rate', '5_ATAW_rate']
X = pd.get_dummies(data_table[features])
Y = data[['FTR']].to_numpy().ravel()

Y = sklearn.preprocessing.OneHotEncoder(Y)
print(Y)
X_imputed = SimpleImputer().fit_transform(X)

In [ ]:
trn_ssn = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
trn_ssn_len = len(trn_ssn)
tst_ssn = [2016,2017,2018] 
tst_ssn_len = len(tst_ssn)



test_size = float(tst_ssn_len)/(tst_ssn_len+trn_ssn_len)
#Split X and Y into training and Test Sets
x_train, x_test, y_train, y_test = train_test_split(data, Y, shuffle=False, test_size=test_size)

#Setup XY to have 10 game steps
x_train = reshape_to_inputshape(x_train,trn_ssn)

y_train = reshape_to_inputshape(y_train,trn_ssn)
y_train = np.moveaxis(y_train, 0, 1)
x_test = reshape_to_inputshape(x_test,tst_ssn)
y_test = reshape_to_inputshape(y_test,tst_ssn)
y_test = np.moveaxis(y_test, 0, 1)

Tx= x_train.shape[1] #Time steps
Ty= y_train.shape[0] #Time Steps
num_features = x_train.shape[2] #Features per step

inputs = tf.keras.Input(shape=(Tx, num_features))

In [ ]:
lst = tf.keras.models.Sequential([
    # tf.keras.Input(shape=(Tx, num_features)),
    # k.layers.Lambda(lambda z: inputs[:, 1, :]),
    k.layers.Flatten(),
    k.layers.Dense(4000),
    k.layers.Dense(240),
    k.layers.Dense(3, activation='softmax')
])

In [ ]:
lst.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001)
#    ,metrics=[tf.keras.metrics.Accuracy()]
)
lst(x_train)
lst.summary()